In [1]:
from yugiquery import *
header('Bandai')

<div align='center'>
    <br>
    <!-- Pre Style needed to fix HTML formatting -->
    <pre style="line-height: var(--jp-code-line-height); font-family: var(--jp-code-font-family)">
    ██    ██ ██    ██  ██████  ██  ██████  ██    ██ ███████ ██████  ██    ██ 
     ██  ██  ██    ██ ██       ██ ██    ██ ██    ██ ██      ██   ██  ██  ██  
      ████   ██    ██ ██   ███ ██ ██    ██ ██    ██ █████   ██████    ████   
       ██    ██    ██ ██    ██ ██ ██ ▄▄ ██ ██    ██ ██      ██   ██    ██    
       ██     ██████   ██████  ██  ██████   ██████  ███████ ██   ██    ██    
                                      ▀▀                                     
    </pre>
</div>
<div align='right'>
    Bandai - Execution started 22/02/2023 22:24 UTC <br>
    By <b>Guilherme Ruiz</b>
</div>

---

# Data aquisition

## Fetch online data

In [2]:
timestamp = pd.Timestamp.now()

In [3]:
bandai_df = fetch_bandai()

120 results


# Check changes

## Load previous data

In [4]:
# Get list of files
files_list = sorted(glob.glob('../data/all_bandai_*.csv'), key=os.path.getctime, reverse=True)
# Get second newest file if exist
if files_list:
    # Load csv avoiding converting "NA" to NaN
    previous_df = pd.read_csv(files_list[0], dtype=object, keep_default_na=False, na_values=[''])
    # Force dtypes to match current df
    previous_df = previous_df.astype(bandai_df[previous_df.columns.intersection(bandai_df.columns)].dtypes.to_dict())
    previous_ts = pd.to_datetime(os.path.basename(files_list[0]).split('_')[-1].rstrip('.csv'))
    print('File loaded')
else:
    previous_df = None
    print('No older files')

File loaded

## Generate changelog

In [5]:
if previous_df is None:
    changelog=None
    print('Skipped')
else:
    changelog = generate_changelog(previous_df, bandai_df, col = 'Name')
    if not changelog.empty:
        display(changelog)
        changelog.to_csv(f'../data/bandai_changelog_{previous_ts.isoformat(timespec="minutes")}_{timestamp.isoformat(timespec="minutes")}.csv', index = True)
        print('Changelog saved')

No changes

## Save data

In [6]:
if changelog is not None and changelog.empty:
    print('No changes. New data not saved')
else:
    bandai_df.to_csv(f'../data/all_bandai_{timestamp.isoformat(timespec="minutes")}.csv', index=False)
    print('Data saved')

No changes. New data not saved

# Data visualization

In [7]:
bandai_df

,Name,Card type,Monster type,DEF,Set,Card number,Ability,Rule,ATK,Level,Page name,Page URL
0,Aphrodite,Monster Card,White Magic,2300,Yu-Gi-Oh! Bandai OCG: 1st Generation,22,NaN,4,1000,5,Aphrodite,https://yugipedia.com/wiki/Aphrodite
1,Armored Basic Insect with Laser Cannon,Monster Card,Insect,1300,Yu-Gi-Oh! Bandai OCG: 1st Generation,17,NaN,8,2400,6,Armored Basic Insect with Laser Cannon,https://yugipedia.com/wiki/Armored_Basic_Insec...
2,Armored Lizard,Monster Card,Beast-Warrior,2000,Yu-Gi-Oh! Bandai OCG: 2nd Generation,73,NaN,6,1800,5,Armored Lizard (Bandai),https://yugipedia.com/wiki/Armored_Lizard_(Ban...
3,Armored Zombie,Monster Card,Fiend,0,Yu-Gi-Oh! Bandai OCG: 2nd Generation,80,NaN,11,2400,4,Armored Zombie (Bandai),https://yugipedia.com/wiki/Armored_Zombie_(Ban...
4,Axe Raider,Monster Card,Warrior,1500,Yu-Gi-Oh! Bandai OCG: 2nd Generation,60,NaN,7,2300,6,Axe Raider (Bandai),https://yugipedia.com/wiki/Axe_Raider_(Bandai)
...,...,...,...,...,...,...,...,...,...,...,...,...
115,The Wicked Worm Beast,Monster Card,Fiend,600,Yu-Gi-Oh! Bandai OCG: 3rd Generation,94,NaN,5,2400,5,The Wicked Worm Beast (Bandai),https://yugipedia.com/wiki/The_Wicked_Worm_Bea...
116,"Winged Dragon, Guardian of the Fortress #1",Monster Card,Dragon,1600,Yu-Gi-Oh! Bandai OCG: 1st Generation,23,NaN,5,1700,4,"Winged Dragon, Guardian of the Fortress 1 (Ban...","https://yugipedia.com/wiki/Winged_Dragon,_Guar..."
117,Yellow Hecate,Monster Card,Black Magic,1500,Yu-Gi-Oh! Bandai OCG: 3rd Generation,98,NaN,NaN,1500,4,Yellow Hecate,https://yugipedia.com/wiki/Yellow_Hecate
118,Yugi Muto,Character Card,NaN,NaN,Yu-Gi-Oh! Bandai OCG: 1st Generation,1,Look Ahead,1,NaN,6,Yugi Muto (Bandai),https://yugipedia.com/wiki/Yugi_Muto_(Bandai)


## Card type

In [8]:
bandai_df['Card type'].nunique()

5

## ATK

In [9]:
bandai_df['ATK'].value_counts()

1800    10
2000    10
1500    10
2400     9
2300     8
2500     5
200      5
2100     4
2600     4
1000     3
800      3
600      2
2200     2
1200     2
1300     2
1600     2
3000     2
1400     2
500      2
100      1
4500     1
1900     1
3200     1
400      1
3500     1
900      1
1700     1
Name: ATK, dtype: int64

In [10]:
bandai_df['ATK'].value_counts()

1800    10
2000    10
1500    10
2400     9
2300     8
2500     5
200      5
2100     4
2600     4
1000     3
800      3
600      2
2200     2
1200     2
1300     2
1600     2
3000     2
1400     2
500      2
100      1
4500     1
1900     1
3200     1
400      1
3500     1
900      1
1700     1
Name: ATK, dtype: int64

## DEF

In [11]:
bandai_df['DEF'].value_counts()

2000    9
1500    8
2200    7
1600    6
2500    5
300     5
2400    5
600     5
2300    5
1000    4
700     4
2100    4
1800    3
400     3
1300    3
800     3
2600    2
1400    2
900     2
0       2
1900    2
1700    1
3000    1
200     1
3500    1
2800    1
500     1
Name: DEF, dtype: int64

In [12]:
bandai_df['DEF'].value_counts()

2000    9
1500    8
2200    7
1600    6
2500    5
300     5
2400    5
600     5
2300    5
1000    4
700     4
2100    4
1800    3
400     3
1300    3
800     3
2600    2
1400    2
900     2
0       2
1900    2
1700    1
3000    1
200     1
3500    1
2800    1
500     1
Name: DEF, dtype: int64

In [13]:
pd.crosstab(bandai_df['DEF'],bandai_df['ATK'])

ATK,100,1000,1200,1300,1400,1500,1600,1700,1800,1900,200,2000,2100,2200,2300,2400,2500,2600,3000,3200,3500,400,4500,500,600,800,900
DEF,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
1000,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
1300,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0
1400,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1500,0,0,0,0,0,3,0,0,0,0,0,2,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0
1600,0,0,0,0,0,0,0,1,0,0,0,1,2,0,1,0,0,1,0,0,0,0,0,0,0,0,0
1700,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1800,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1900,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
pd.to_numeric(bandai_df['ATK'].dropna(), errors = 'ignore').astype(int)

0      1000
1      2400
2      1800
3      2400
4      2300
       ... 
112    1500
113    1500
115    2400
116    1700
117    1500
Name: ATK, Length: 95, dtype: int64

## Level

In [15]:
bandai_df['Level'].nunique()

8

In [16]:
bandai_df['Level'].value_counts()

5    34
6    23
4    17
7    10
8     7
1     5
2     4
3     3
Name: Level, dtype: int64

### By ATK

In [17]:
pd.crosstab(bandai_df['Level'],bandai_df['ATK'])

ATK,100,1000,1200,1300,1400,1500,1600,1700,1800,1900,200,2000,2100,2200,2300,2400,2500,2600,3000,3200,3500,400,4500,500,600,800,900
Level,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0
3,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,0,0,1,1,4,1,1,3,0,0,2,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0
5,0,2,1,1,1,3,0,0,4,0,0,7,3,1,1,3,2,0,0,0,0,1,0,0,0,2,1
6,0,0,1,0,0,2,1,0,1,1,0,1,1,1,7,3,1,1,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2,2,3,0,1,0,0,0,0,0,0,0
8,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,1,0,1,0,0,0,0


### By DEF

In [18]:
pd.crosstab(bandai_df['Level'],bandai_df['DEF'])

DEF,0,1000,1300,1400,1500,1600,1700,1800,1900,200,2000,2100,2200,2300,2400,2500,2600,2800,300,3000,3500,400,500,600,700,800,900
Level,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,1,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
4,1,1,0,0,3,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2,3,1,1
5,0,0,2,2,2,3,0,1,1,0,6,3,3,2,1,1,0,0,0,0,0,1,1,2,1,1,0
6,0,1,1,0,3,1,1,0,1,0,1,1,3,2,2,1,0,1,0,0,1,0,0,0,0,0,1
7,0,0,0,0,0,1,0,1,0,0,1,0,0,1,2,1,2,0,0,0,0,0,0,0,0,0,0
8,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,2,0,0,0,1,0,0,0,0,0,0,0


## Monster type

In [19]:
bandai_df['Monster type'].nunique()

15

In [20]:
bandai_df['Monster type'].value_counts()

Fiend              18
Dragon             13
Warrior            12
Black Magic        12
Insect              7
White Magic         6
Beast-Warrior       5
Dinosaur            5
Beast               4
Winged Beast        3
Illusion Magic      2
Sea Beast           2
Dragon Magic        1
Sea Serpent         1
Thunder             1
Name: Monster type, dtype: int64

# Epilogue

In [21]:
benchmark('bandai',timestamp)

In [22]:
footer()

<div align='right'>
    Execution completed 22/02/2023 22:24 UTC
</div>

## HTML export

In [23]:
# Save notebook on disck before generating HTML report
save_notebook()

Notebook saved to disk

In [24]:
! jupyter nbconvert Bandai.ipynb --output-dir='../' --to=HTML --TagRemovePreprocessor.enabled=True --TagRemovePreprocessor.remove_cell_tags='exclude' --TemplateExporter.exclude_input=True --TemplateExporter.exclude_input_prompt=True --TemplateExporter.exclude_output_prompt=True

[NbConvertApp] Converting notebook Bandai.ipynb to HTML


[NbConvertApp] Writing 624071 bytes to ../Bandai.html


## Git

In [ ]:
! git add "../*[Bb]andai*"

In [ ]:
! git commit -m {"'Bandai update-" + timestamp.isoformat() + "'"}